In [71]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
###################### Matplotlib #######################
# import matplotlib pyplot commands
from matplotlib.pyplot import *

import matplotlib
matplotlib.style.use('ggplot')

# Show Plots in the Notebook
%matplotlib inline


In [72]:
df = pd.read_csv(r"C:\Users\dustin\Desktop\AuditModeledMasterDataSheet wVentilation&Costs 3-26-2013 - Updated Schools Graph.csv", header=0, na_values=[' -  ',' -   ','#DIV/0!'])
df.head()

,Com_AkwarmMetadataID,bi_LocID,Com_Buildinginputid,bi_Addr,bi_City,bi_BldgName,bi_CreateDate,bi_AuditDate,bi_YearBuilt,bi_yearlastremodel,...,Thermal BTU/HDD/Year),Annual Cost Eff. Energy Savings,Installed Cost,Simple payback,Has retrofit info?,Cost eff energy savings / SF,% cost eff energy savings,NPV savings,Unnamed: 374,Unnamed: 375
0,157,88169,153,P O Box 109,Sleetmute,Sleetmute Water Treatment Plant,40982.46939,2011-04-26 10:09:18.0000000 +00:00,1990,0,...,"5,528","$3,284","$5,700",1.7,1,$4.74,17%,"$48,897",NaN,NaN
1,10,85519,6,P O Box 100,Shaktoolik,Tribal Office Building,9/30/2011,2011-04-07 09:36:10.0000000 +00:00,1975,0,...,"10,996","$1,518","$3,371",2.2,1,$1.43,28%,"$22,413",NaN,NaN
2,496,83418,442,1710 30th,Fairbanks,Fire Training Center,7/24/2012,2011-08-16 14:23:13.7855186 -08:00,1981,0,...,"22,238","$1,553","$4,629",3,1,$0.41,20%,"$18,124",NaN,NaN
3,603,120466,549,Lot 3 ANCSA 14(c) Plat 90-2,False Pass,False Pass City Hall,7/27/2012,2012-01-24 13:15:27.4991289 -09:00,1985,0,...,"12,987",$858,"$1,600",1.9,1,$0.35,14%,"$10,633",NaN,NaN
4,11,85537,16,P O Box 10,Shaktoolik,Water Treatment Plant,11/2/2011,2011-04-04 08:10:42.0000000 +00:00,1977,0,...,"16,427","$11,020","$3,500",0.3,1,$5.22,54%,"$196,749",NaN,NaN


In [73]:
df.dtypes

Com_AkwarmMetadataID                 int64
bi_LocID                           float64
Com_Buildinginputid                  int64
bi_Addr                             object
bi_City                             object
bi_BldgName                         object
bi_CreateDate                       object
bi_AuditDate                        object
bi_YearBuilt                         int64
bi_yearlastremodel                   int64
bi_Reslivingunits                    int64
bi_BldgOccups                       object
Com_Buildingtypeid                   int64
BuildingType                        object
ChangedBuildingTypeID               object
Otherbuildingtypedescription        object
Usage Types Changes                 object
Research Usage Types:               object
IsEducation?                       float64
Electricutilityid                    int64
Elec_Utility_Name                   object
IsCommercial                         int64
Gasutilityid                         int64
Gas_Utility

In [74]:
clean_df = df[['Com_AkwarmMetadataID', 'Research Usage Types:','Total Vent AirExch (ft3/yr)', 
               'Vent / SQFT / YR', 'Vent / Envelope Area / YR', 'ThermalBTU/YR', 'bi_Addr', 'bi_City', 
               'bi_BldgName', 'Envelope U-value', 'EnvelopeArea:', 'HDD(FromWhitePaper)', 'PrimaryFuel_C/MMBTU',
              'AnnualEnergyMMBTUs', 'RegionName', 'Building Age:','BEST SQFT']]
clean_df.head()

,Com_AkwarmMetadataID,Research Usage Types:,Total Vent AirExch (ft3/yr),Vent / SQFT / YR,Vent / Envelope Area / YR,ThermalBTU/YR,bi_Addr,bi_City,bi_BldgName,Envelope U-value,EnvelopeArea:,HDD(FromWhitePaper),PrimaryFuel_C/MMBTU,AnnualEnergyMMBTUs,RegionName,Building Age:,BEST SQFT
0,157,Washeteria / Water Plant,NaN,NaN,NaN,73733315,P O Box 109,Sleetmute,Sleetmute Water Treatment Plant,0.049,2312,13339,45.45,380,Calista,22,693
1,10,Office_Less_Than_20000,NaN,NaN,NaN,153055389,P O Box 100,Shaktoolik,Tribal Office Building,0.082,3523,13919,25.91,177,Bering Straits Native Corporation,37,1064
2,496,Education_Single,312000,82,285,317419830,1710 30th,Fairbanks,Fire Training Center,0.051,11758,14274,18.41,345,Doyon - Fairbanks North Star Borough only,31,3796
3,603,Office_Less_Than_20000,NaN,NaN,NaN,126399681,Lot 3 ANCSA 14(c) Plat 90-2,False Pass,False Pass City Hall,0.047,6632,9733,28.12,145,Aleut,27,2448
4,11,Washeteria / Water Plant,NaN,NaN,NaN,228648600,P O Box 10,Shaktoolik,Water Treatment Plant,0.082,6638,13919,25.91,688,Bering Straits Native Corporation,35,2112


In [75]:
clean_df = clean_df.rename(columns = {'Research Usage Types:':'Usage_Type',
                                      'Total Vent AirExch (ft3/yr)':'TotalVent',
                                      'Envelope U-value':'TotalUvalue',
                                      'BEST SQFT':'sqft',
                                      'HDD(FromWhitePaper)':'HDD',
                                      'Building Age:':'age',
                                      'ThermalBTU/YR':'AnnThermalBTU',
                                      'EnvelopeArea:':'EnvelopeArea'
                                     })
clean_df.head()

,Com_AkwarmMetadataID,Usage_Type,TotalVent,Vent / SQFT / YR,Vent / Envelope Area / YR,AnnThermalBTU,bi_Addr,bi_City,bi_BldgName,TotalUvalue,EnvelopeArea,HDD,PrimaryFuel_C/MMBTU,AnnualEnergyMMBTUs,RegionName,age,sqft
0,157,Washeteria / Water Plant,NaN,NaN,NaN,73733315,P O Box 109,Sleetmute,Sleetmute Water Treatment Plant,0.049,2312,13339,45.45,380,Calista,22,693
1,10,Office_Less_Than_20000,NaN,NaN,NaN,153055389,P O Box 100,Shaktoolik,Tribal Office Building,0.082,3523,13919,25.91,177,Bering Straits Native Corporation,37,1064
2,496,Education_Single,312000,82,285,317419830,1710 30th,Fairbanks,Fire Training Center,0.051,11758,14274,18.41,345,Doyon - Fairbanks North Star Borough only,31,3796
3,603,Office_Less_Than_20000,NaN,NaN,NaN,126399681,Lot 3 ANCSA 14(c) Plat 90-2,False Pass,False Pass City Hall,0.047,6632,9733,28.12,145,Aleut,27,2448
4,11,Washeteria / Water Plant,NaN,NaN,NaN,228648600,P O Box 10,Shaktoolik,Water Treatment Plant,0.082,6638,13919,25.91,688,Bering Straits Native Corporation,35,2112


In [76]:
clean_df.dtypes

Com_AkwarmMetadataID           int64
Usage_Type                    object
TotalVent                    float64
Vent / SQFT / YR             float64
Vent / Envelope Area / YR    float64
AnnThermalBTU                  int64
bi_Addr                       object
bi_City                       object
bi_BldgName                   object
TotalUvalue                  float64
EnvelopeArea                 float64
HDD                          float64
PrimaryFuel_C/MMBTU          float64
AnnualEnergyMMBTUs             int64
RegionName                    object
age                            int64
sqft                           int64
dtype: object

In [84]:
clean_df['ThermalKBTU'] = clean_df['AnnThermalBTU']/1000
clean_df.head()

,Com_AkwarmMetadataID,Usage_Type,TotalVent,Vent / SQFT / YR,Vent / Envelope Area / YR,AnnThermalBTU,bi_Addr,bi_City,bi_BldgName,TotalUvalue,EnvelopeArea,HDD,PrimaryFuel_C/MMBTU,AnnualEnergyMMBTUs,RegionName,age,sqft,ThermalMMBTU,ThermalKBTU
0,157,Washeteria / Water Plant,NaN,NaN,NaN,73733315,P O Box 109,Sleetmute,Sleetmute Water Treatment Plant,0.049,2312,13339,45.45,380,Calista,22,693,73.733315,73733.315
1,10,Office_Less_Than_20000,NaN,NaN,NaN,153055389,P O Box 100,Shaktoolik,Tribal Office Building,0.082,3523,13919,25.91,177,Bering Straits Native Corporation,37,1064,153.055389,153055.389
2,496,Education_Single,312000,82,285,317419830,1710 30th,Fairbanks,Fire Training Center,0.051,11758,14274,18.41,345,Doyon - Fairbanks North Star Borough only,31,3796,317.419830,317419.830
3,603,Office_Less_Than_20000,NaN,NaN,NaN,126399681,Lot 3 ANCSA 14(c) Plat 90-2,False Pass,False Pass City Hall,0.047,6632,9733,28.12,145,Aleut,27,2448,126.399681,126399.681
4,11,Washeteria / Water Plant,NaN,NaN,NaN,228648600,P O Box 10,Shaktoolik,Water Treatment Plant,0.082,6638,13919,25.91,688,Bering Straits Native Corporation,35,2112,228.648600,228648.600


In [85]:
est = smf.ols(formula='ThermalKBTU ~ HDD + EnvelopeArea + TotalUvalue + TotalVent', data=clean_df).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            ThermalKBTU   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     216.6
Date:                Tue, 15 Sep 2015   Prob (F-statistic):           2.85e-84
Time:                        11:16:49   Log-Likelihood:                -4575.5
No. Observations:                 284   AIC:                             9161.
Df Residuals:                     279   BIC:                             9179.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept    -4.168e+06   9.27e+05     -4.496      0.000     -5.99e+06 -2.34e+06
HDD            268.8950     53.168      5.057      0.000       164.234   373.556
EnvelopeArea    26.8161      1.876     14.297      0.000        23.124    30.508
TotalUvalue   1.685e+07   5.68e+06      2.967      0.003      5.67e+06   2.8e+07
TotalVent        0.0004   2.51e-05     15.661      0.000         0.000     0.000
==============================================================================
Omnibus:                      333.241   Durbin-Watson:                   2.178
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            26836.999
Skew:                           4.923   Prob(JB):                         0.00
Kurtosis:                      49.594   Cond. No.                     3.03e+11
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.03e+11. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [86]:
vent_est = smf.ols(formula='ThermalKBTU ~ TotalVent', data=clean_df).fit()
vent_est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            ThermalKBTU   R-squared:                       0.564
Model:                            OLS   Adj. R-squared:                  0.563
Method:                 Least Squares   F-statistic:                     366.6
Date:                Tue, 15 Sep 2015   Prob (F-statistic):           5.44e-53
Time:                        12:03:13   Log-Likelihood:                -4674.3
No. Observations:                 285   AIC:                             9353.
Df Residuals:                     283   BIC:                             9360.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   2.129e+06   2.15e+05      9.886      0.000      1.71e+06  2.55e+06
TotalVent      0.0005   2.83e-05     19.147      0.000         0.000     0.001
==============================================================================
Omnibus:                      213.805   Durbin-Watson:                   1.707
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4514.785
Skew:                           2.757   Prob(JB):                         0.00
Kurtosis:                      21.702   Cond. No.                     8.60e+09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.6e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [88]:
clean_df['Usage_Type'].unique()

array(['Washeteria / Water Plant', 'Office_Less_Than_20000',
       'Education_Single', 'Public_Order', 'Health Clinic',
       'Maintenance / Shop', 'Education_Less_Than_30000',
       'Athletic_Facility', 'Public_Assembly',
       'Education_Greater_Than_30000', 'Warehousing',
       'Office_20000_To_50000', 'Terminals (Airport, Bus, Harbor, Train)',
       'Office_Greater_Than_50000', 'Residential_Multi_Fam_5_Plus_Units',
       'Hospital', 'Pool', 'Other', 'Accommodations_Hotel'], dtype=object)

In [89]:
schools = clean_df.query("Usage_Type == 'Education_Greater_Than_30000'")
schools.head()

,Com_AkwarmMetadataID,Usage_Type,TotalVent,Vent / SQFT / YR,Vent / Envelope Area / YR,AnnThermalBTU,bi_Addr,bi_City,bi_BldgName,TotalUvalue,EnvelopeArea,HDD,PrimaryFuel_C/MMBTU,AnnualEnergyMMBTUs,RegionName,age,sqft,ThermalMMBTU,ThermalKBTU
38,429,Education_Greater_Than_30000,3531713858,53215,217346,6157784390,11700 Gregory Road,Anchorage,Bowman Elementary School,0.050,174723,10816,9.40,7983,Cook Inlet Regional (CIRI) - Anchorage Borough...,21,66367,6157.784390,6157784.390
39,430,Education_Greater_Than_30000,2308146856,40272,178697,5615185000,3101 West 88th Avenue,Anchorage,Chinook Elementary,0.059,138888,10816,9.45,7007,Cook Inlet Regional (CIRI) - Anchorage Borough...,44,57314,5615.185000,5615185.000
41,420,Education_Greater_Than_30000,NaN,NaN,NaN,2295817140,1000 School Road,Nondalton,Nondalton K-12,0.046,37558,11130,37.68,2454,Bristol Bay Native Corporation,34,21744,2295.817140,2295817.140
56,452,Education_Greater_Than_30000,769007342,19136,97827,3641970004,705 Frontage Road,Kenai,Kenai Elementary,0.112,84526,11395,8.95,5196,Cook Inlet Regional (CIRI) - outside Anchorage...,63,40187,3641.970004,3641970.004
72,426,Education_Greater_Than_30000,2095778367,56548,314971,1579631000,Main Elem School Drive,Kodiak,Main Elementary School,0.047,71547,8539,16.09,2704,Koniag,28,37062,1579.631000,1579631.000


In [91]:
schools.shape

(104, 19)

In [92]:
schools_est = smf.ols(formula='ThermalKBTU ~ HDD + EnvelopeArea + TotalUvalue + TotalVent', data=schools).fit()
schools_est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            ThermalKBTU   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.783
Method:                 Least Squares   F-statistic:                     90.17
Date:                Tue, 15 Sep 2015   Prob (F-statistic):           1.75e-31
Time:                        12:07:26   Log-Likelihood:                -1618.1
No. Observations:                 100   AIC:                             3246.
Df Residuals:                      95   BIC:                             3259.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept    -3.238e+06   1.88e+06     -1.726      0.088     -6.96e+06  4.87e+05
HDD            201.4390    111.315      1.810      0.074       -19.550   422.428
EnvelopeArea    25.6131      3.132      8.178      0.000        19.395    31.831
TotalUvalue   1.257e+07   1.11e+07      1.131      0.261     -9.49e+06  3.46e+07
TotalVent        0.0004    4.8e-05      8.856      0.000         0.000     0.001
==============================================================================
Omnibus:                       38.934   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              116.462
Skew:                           1.329   Prob(JB):                     5.14e-26
Kurtosis:                       7.571   Cond. No.                     3.79e+11
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.79e+11. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [93]:
print schools_est.params

Intercept       -3237709.209303
HDD                  201.439031
EnvelopeArea          25.613069
TotalUvalue     12569424.652021
TotalVent              0.000425
dtype: float64


0.791511777325
